In [72]:
import pandas as pd
import numpy as np

In [218]:
# def getUtilityMatrix(csvfile):
#     df = pd.read_csv(csvfile,usecols=['userid','movieid','rating'])
#     num_users = len(df['userId'].unique())
#     num_movies = len(df['movieId'].unique())
#     movieIdlist = df['movieId'].unique().tolist()
#     movieIdlist.sort()
#     utility_mat = np.zeros((num_users,num_movies))
#     for row in df:
#         utility_mat[row['userid']-1][movieIdlist.index(row['movieid'])] = row['rating']
#     return utility_mat

# def getMetadataMatrix(csv):
    
    
def getUserFeatureMatrix(ratingcsv,moviefeaturecsv,maxUsers):
    df = pd.read_csv(moviefeaturecsv)
    df.fillna(0,inplace = True)
    df.set_index('id',inplace=True)
    
    df2 = pd.read_csv(ratingcsv,usecols=['userId','movieId','rating'])
    df2.fillna(0,inplace = True)
    df2['userId'] = pd.to_numeric(df2['userId'])
    df2['movieId'] = pd.to_numeric(df2['movieId'])
    df2['rating'] = pd.to_numeric(df2['rating'])
    
    commonMovieIds = df.index.intersection(df2['movieId'].unique(),sort = None).tolist()
    ratingdf = df2[df2['movieId'].isin(commonMovieIds)]
    featuresdf = df[df.index.isin(commonMovieIds)]
    
    
    #dropping users
    users = df2['userId'].unique()
    users.sort()
    users = users[:maxUsers]
    print(max(users))
    ratingdf = ratingdf[ratingdf['userId'].isin(users)]
    
    num_users = len(users)
    num_movies = len(commonMovieIds)    
    print(num_users)
    print(num_movies)
    utility_mat = np.zeros((num_users,num_movies))
    for _,row in ratingdf.iterrows():            
        utility_mat[int(row['userId']-1)][commonMovieIds.index(int(row['movieId']))] = row['rating']
    
    movieFeaturesMat = featuresdf.as_matrix()
    
    userFeaturesMat = np.matmul(utility_mat, movieFeaturesMat)
    num_genre = 19
    num_prod_comp = 23
    num_prod_ctry = 32
    num_cast = 13
    
    for i in range(userFeaturesMat.shape[0]):
        genre_sum = np.linalg.norm(userFeaturesMat[i][:num_genre],None) + 0.0001        
        userFeaturesMat[i][:num_genre] = userFeaturesMat[i][:num_genre] /genre_sum
        
        cast_sum = np.linalg.norm(userFeaturesMat[i][num_genre:num_genre + num_cast],None) + 0.0001
        userFeaturesMat[i][num_genre:num_genre + num_cast] = userFeaturesMat[i][num_genre:num_genre + num_cast]/cast_sum

        
        prod_com_sum = np.linalg.norm(userFeaturesMat[i][num_genre + num_cast:num_genre + num_cast + num_prod_comp],None) + 0.0001
        userFeaturesMat[i][num_genre + num_cast:num_genre + num_cast + num_prod_comp] = userFeaturesMat[i][num_genre + num_cast:num_genre + num_cast + num_prod_comp] /prod_com_sum
        
        prod_ctry_sum = np.linalg.norm(userFeaturesMat[i][num_genre + num_cast + num_prod_comp: num_genre + num_cast + num_prod_comp + num_prod_ctry],None) + 0.0001
        userFeaturesMat[i][num_genre + num_cast + num_prod_comp: num_genre + num_cast + num_prod_comp + num_prod_ctry] = userFeaturesMat[i][num_genre + num_cast + num_prod_comp:num_genre + num_cast + num_prod_comp + num_prod_ctry]/prod_ctry_sum        

    userFeatureDF = pd.DataFrame(userFeaturesMat, columns = df.columns.tolist())
    userFeatureDF = userFeatureDF.loc[:, (userFeatureDF != 0).any(axis=0)]
  
        
    utilityDF = pd.DataFrame(utility_mat, columns = commonMovieIds )
    utilityDF = utilityDF.loc[:, (utilityDF != 0).any(axis=0)]
    
    return utilityDF,userFeatureDF
    
    
    
    
    
    
    

In [219]:
a,b = getUserFeatureMatrix("./the-movies-dataset/ratings.csv","./movies_cross_features.csv",50000)

50000
50000
7565


/home/ayush/.local/lib/python2.7/site-packages/ipykernel_launcher.py:46: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


In [220]:
a

,2,3,5,6,11,12,13,14,15,16,...,175291,175331,175427,175553,175555,175587,175791,176037,176077,176143
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [221]:
b

,genre_Drama,genre_Comedy,genre_Thriller,genre_Romance,genre_Action,genre_Horror,genre_Crime,genre_Documentary,genre_Adventure,genre_Science Fiction,...,prod_ctry_History,prod_ctry_Horror,prod_ctry_Music,prod_ctry_Mystery,prod_ctry_Romance,prod_ctry_Science Fiction,prod_ctry_TV Movie,prod_ctry_Thriller,prod_ctry_War,prod_ctry_Western
0,0.838537,0.431974,0.000000,0.254102,0.088936,0.000000,0.114346,0.127051,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.025410,0.254102,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.654652,0.305504,0.174574,0.261861,0.327326,0.174574,0.065465,0.000000,0.305504,0.087287,...,0.152752,0.174574,0.021822,0.152752,0.261861,0.087287,0.000000,0.174574,0.065465,0.000000
2,0.602169,0.566747,0.177108,0.354217,0.212530,0.000000,0.283374,0.000000,0.000000,0.106265,...,0.000000,0.000000,0.000000,0.106265,0.354217,0.106265,0.000000,0.177108,0.000000,0.000000
3,0.681160,0.348887,0.182750,0.315659,0.232591,0.066455,0.282432,0.066455,0.199364,0.132909,...,0.000000,0.066455,0.000000,0.116296,0.315659,0.132909,0.000000,0.182750,0.000000,0.166137
4,0.708321,0.262341,0.341043,0.262341,0.209873,0.131170,0.262341,0.000000,0.052468,0.288575,...,0.000000,0.131170,0.000000,0.131170,0.262341,0.288575,0.000000,0.341043,0.000000,0.104936
5,0.000000,0.372998,0.000000,0.372998,0.279749,0.000000,0.000000,0.000000,0.279749,0.652747,...,0.000000,0.000000,0.000000,0.000000,0.372998,0.652747,0.000000,0.000000,0.000000,0.000000
6,0.763697,0.412809,0.237365,0.082562,0.309607,0.051601,0.051601,0.134163,0.185764,0.051601,...,0.061803,0.051503,0.041202,0.000000,0.082404,0.051503,0.061803,0.236912,0.000000,0.103005
7,0.725494,0.311895,0.352577,0.183069,0.277993,0.149167,0.250872,0.000000,0.135606,0.115265,...,0.054243,0.149167,0.047462,0.000000,0.183069,0.115265,0.000000,0.352577,0.033902,0.027121
8,0.782964,0.449480,0.106328,0.106328,0.178825,0.000000,0.231989,0.000000,0.135327,0.043498,...,0.033832,0.000000,0.000000,0.125661,0.106328,0.043498,0.000000,0.106328,0.033832,0.096662
9,0.644788,0.423142,0.181347,0.080598,0.261945,0.100748,0.342544,0.000000,0.161197,0.261945,...,0.000000,0.100748,0.000000,0.261945,0.080598,0.261945,0.000000,0.181347,0.000000,0.000000
